# Approche : 

Nous allons utiliser une approche basée sur la distance de Leveishtein, une distance mathématique donnant une mesure de la similarité entre deux chaînes de caractères. Elle est égale au nombre minimal de caractères qu'il faut supprimer, insérer ou remplacer pour passer d’une chaîne à l’autre.

Nous comparons chacune des alertes avec l'ensemble des titres de produits et récupérons le produit avec le plus petit score. 

En parcourant l'ensemble des alertes et en effectuant quelques tests, plusieurs choses nous sautent aux yeux : 
- Plusieurs fois, les clients omettent la marque du produit en mentionnant seulement le modèle, exemple : Galaxy S3 au lieu de Samsung Galaxy S3. 
- Les clients ont tendance à ne pas aller dans le détail du produit en question, exemple : Iphone 7 au lieu de "IPHONE 7 128 GO  NOIR DE JAIS  DÉBLOQUÉ "
- La distance de Leveinshtein met quelques secondes à s'exécuter, jusqu'à quelques minutes si on parcourt tous nos produits.
- Certains clients ont tendance à oublier une partie du nom du produit : "Samsung S5" au lieu de "Samgsung Galaxy S5" 

Pour pallier à ces difficultés, nous avons quelques approches : 
- Nous remarquons que le premier mot des alertes est souvent le plus important, celui qui donne une indication sur la marque et parfois même le produit lui même. Ainsi, nous mettrons une condition (voir code, commentaire (1)) sur la présence de ce mot dans le produit. Si le mot est présent nous calculons la distance, sinon nous donnons un score élevé au produit.
- Nous ne comparerons que des chaines de même taille, nous limiterons la taille de la chaîne des produits à celle de l'article. Exemple : au lieu de comparer "Iphone 7" et  "IPHONE 7 128 GO  NOIR DE JAIS  DÉBLOQUÉ ", nous comparons "Iphone 7" et "IPHONE 7". (commentaire (2))
- Nous calculons la distance entre le produit et l'alerte qu'à partir de l'apparition du premier mot dans le produit. Exemple : Au lieu de comparer  "Galaxy S3" et "Samsung Galaxy S3 Noir 16GB" nous comparons "Galaxy S3" et "Galaxy S3 Noir 16GB". (commentaire (3))
- Nous calculerons une distance en ignorant le second mot. (Commentaire (4))


Nous modifierons aussi les textes en mettant le tout en majuscule et en retirant quelques caractères spéciaux

# Imports

In [1]:
import pandas as pd 
import numpy as np 
import distance
import re
from IPython.display import display
import glob

# Observation de nos données 

On regarde le fichier français : 

In [2]:
df_products = pd.read_csv("HOME_MISSION_SHARE/Recomendations/products_data_fr_fr_2017-11-14_150627.187511.csv", \
                          delimiter=";", encoding="latin1")
df_alerts = pd.read_csv("HOME_MISSION_SHARE/Recomendations/alerts_fr_fr_2017-11-14_151447.572712.csv", \
                        delimiter=";", encoding="latin1")

Les produits : 

In [3]:
df_products.head()

,title_fr_fr,id,description_fr_fr
0,HTC Windows 8S 4 Go - Noir - Bouygues,222,"\tSmartphone sous Android 2.1, le WildFire vie..."
1,Nokia Lumia 1020 32 Go - Blanc - SFR,228,Compatible 4G - Windows Phone 8 - Ecran AMOLED...
2,iPhone 4 16 Go - Noir - Débloqué,1178,<p><b>iPhone 4 16 Go - Noir Débloqué</b></p>\...
3,iPhone 4 16 Go - Blanc - Débloqué,1179,<strong> Le mobile sera livré sous le dernier ...
4,iPhone 4 32 Go - Noir - Débloqué,1180,<p><b>iPhone 4 32 Go - Noir - Débloqué</b></p>...


Les alertes : 

In [4]:
df_alerts

,pk,product,criterias,country_code,creation_date
0,30254,IPAD MINI 4 32 GO WIFI + 4g Or Débloqué,NaN,fr_fr,2017-11-14 13:09:03+00:00
1,30252,EXPRESSO À CAPSULES Dolce Gusto Krups Yy1786fd,NaN,fr_fr,2017-11-14 12:18:30+00:00
2,30244,ACER SWIFT Sf314-51-39zj 14 Core I3-6006u 2 Gh...,NaN,fr_fr,2017-11-14 11:43:47+00:00
3,30226,IPAD 4 128 GO WIFI NOIR,NaN,fr_fr,2017-11-14 07:52:01+00:00
4,30217,COMPTEUR DE FANS Facebook 5 Chiffres,NaN,fr_fr,2017-11-14 06:27:33+00:00
5,30216,ASUS ZENFONE 2 SELFIE 32 Go Bleu Débloqué,NaN,fr_fr,2017-11-13 23:34:03+00:00
6,30207,DORO 8031 8 GO NOIR Débloqué,NaN,fr_fr,2017-11-13 22:02:28+00:00
7,30199,VENTILATEUR DYSON Cool Am07,NaN,fr_fr,2017-11-13 20:39:24+00:00
8,30198,LG G6 32 GO NOIR Débloqué,NaN,fr_fr,2017-11-13 20:42:47+00:00
9,30197,"IPAD MINI 4 7,9'' 32 GO Wifi Argent",NaN,fr_fr,2017-11-13 20:32:07+00:00


Quelques produits peuvent poser problème : 
- "iphone7" : il manque l'espace
- "xperia", "galaxy" : il manque le nom de marque 
- Concaténer les informations dans critères

Définition d'une fonction retournant la distance de levenshtein entre deux chaines de caractères : 

In [5]:
def compute_levenshtein(product, alert):

    
    #Split à la virgule pour les listes de produits : on ne prend que le premier
    if ", " in alert: 
        alert = alert.split(',')[0]
    
    
    #On retire les espaces multiples
    product = re.sub('\s+', ' ', product).strip()
    alert = re.sub('\s+', ' ', alert).strip()
    
    
    #Commentaire (1) : Le premier mot est toujours le plus important dans les alertes
    if alert.split()[0] in product:
        
        #Commentaire (3) : Nous chercons le mot de l'alerte pour commencer la chaine
        product = product[product.find(alert.split()[0]):]
        
        product_ignore_scnd_word = product.split(" ")
        product_ignore_scnd_word = " ".join([product_ignore_scnd_word[0]] + product_ignore_scnd_word[2:])
        
        #Commentaire (2) : Nous calculons la distance pour des chaines de même taille
        #Commentaire (4) : Nous prenons le minimum entre la distance des deux chaine et les
        # deux chaines privées du second mot
        return min(distance.levenshtein(product[:len(alert)], alert), \
                   distance.levenshtein(product_ignore_scnd_word[:len(alert)], alert))
    else :
        return 999

In [6]:
files = (glob.glob("HOME_MISSION_SHARE/Recomendations/*.csv"))

In [7]:
#On itère sur tous nos fichiers :
for i in range(0, int(len(files)/2)):
    #import des tables : 
    df_alerts = pd.read_csv(files[i], delimiter=";", encoding="latin1")
    df_products = pd.read_csv(files[i+int(len(files)/2)], delimiter=";", encoding="latin1")
    
    #Quelques modifications sur les chaines de caractères
    df_alerts["product"] = df_alerts["product"].map(lambda x : x.upper().replace('-','')) 
    
    #On cherche le titre du produit (il n'a pas toujours la même place en fonction des langues)
    for col in df_products.columns:
        if "title" in col:
            title = col 
    
    df_products[title] = df_products[title].map(lambda x : x.upper().replace('-','')) 
    
    #On crée une nouvelle colonne :
    df_alerts["prod_correspondance"] = [[] for i in range(df_alerts.shape[0])]
    
    for j, prod in enumerate(df_alerts["product"]):
        #On donne un score de matching entre l'alerte et chaque produit du catalogue
        df_products["leveinshtein"] = df_products[title].map(lambda x : compute_levenshtein(x, prod))
        
        if df_products["leveinshtein"].min() == 999:
            df_alerts.iloc[j, -1] = "No corresponding products found !"
        else :
            #On met les 7 top produits correspondants : 
            for item in (df_products.sort_values("leveinshtein")[title].iloc[:7].values):
                df_alerts.iloc[j, -1].append(item)
        
    df_alerts.to_csv("HOME_MISSION_SHARE/Recomendations/alerts_"+title.split("_")[-1]+".csv")
    print("-----------------------------------------------\n")
    print("\t Matching des produits "+ title.split("_")[-1])
    print("\n-----------------------------------------------")
    display(df_alerts[["product", "prod_correspondance"]])

-----------------------------------------------

	 Matching des produits de

-----------------------------------------------


,product,prod_correspondance
0,SAMSUNG TAB3,"[SAMSUNG GALAXY TAB 3 LITE 7 8 GB SCHWARZ, S..."
1,SAMSUNG GALAXY S7 EDGE,[SAMSUNG GALAXY S7 EDGEDUAL SIM 32GB SCHWARZ ...
2,CANON EOS 1300D,"[REFLEX CANON EOS 1000D SCHWARZ, SLR CANON ..."
3,SAMSUNG J5 (6),"[SAMSUNG GALAXY S5 16 GB G900F 4G GOLD SFR, ..."
4,SAMSUNG GALAXY 10 ZOLL,[SAMSUNG GALAXY S4 ZOOM 8 GB SCHWARZ OHNE V...
5,SAMSUNG J5 2015,[SAMSUNG GALAXY A5 2016 16 GB SCHWARZ OHNE V...
6,MICROSOFT SURFACE PRO,[MICROSOFT SURFACE PRO 3 12 256 GB I5 SCHWARZ...
7,"IPAD PRO 10,5 512GB WLAN/LTE","[IPAD PRO 10,5 512 GB WLAN + LTE SPACEGRAU ..."
8,GALAXY NOTE EDGE 32GB WEISS OHNE VERTRAG,[SAMSUNG GALAXY NOTE EDGE 32GB WEISS OHNE VER...
9,NINTENDO 3DS,"[NINTENDO KONSOLE DS LITE METAL ROSE, NINTEND..."


-----------------------------------------------

	 Matching des produits es

-----------------------------------------------


,product,prod_correspondance
0,MACBOOK PRO 13 CORE I5 2.5 GHZ HDD 750 GB RAM...,[MACBOOK PRO 13 CORE I5 2.5 GHZ HDD 750 GB ...
1,TV LED PHILIPS 43PFT4001/05,"[TV LED PHILIPS 43PFT4001/05, TV LED PHILIPS 5..."
2,LENOVO MOTO G4 PLUS 16 GB NEGRO LIBRE,"[LENOVO MOTO G4 PLUS 16 GB NEGRO LIBRE, LEN..."
3,IPHONE 8 64 GB GRIS ESPACIAL LIBRE,"[IPHONE 8 64 GB GRIS ESPACIAL LIBRE, IPHONE ..."
4,BLACKBERRY BOLD 9900 8 GB NEGRO DESBLOQUEADO,[BLACKBERRY BOLD 9900 8 GB NEGRO DESBLOQUEAD...
5,IPHONE 6S PLUS,"[IPHONE 6S PLUS 32 GB ORO ROSA LIBRE, IPHONE..."
6,IPHONE 6S PLUS,"[IPHONE 6S PLUS 32 GB ORO ROSA LIBRE, IPHONE..."
7,IPHONE 6 PLUS,"[IPHONE 6 PLUS 64 GB GRIS ESPACIAL LIBRE, IP..."
8,IPHONE 6 PLUS,"[IPHONE 6 PLUS 64 GB GRIS ESPACIAL LIBRE, IP..."
9,IPHONE 6S PLUS,"[IPHONE 6S PLUS 32 GB ORO ROSA LIBRE, IPHONE..."


-----------------------------------------------

	 Matching des produits fr

-----------------------------------------------


,product,prod_correspondance
0,IPAD MINI 4 32 GO WIFI + 4G OR DÉBLOQUÉ,"[IPAD MINI 4 32 GO WIFI + 4G OR DÉBLOQUÉ, IPA..."
1,EXPRESSO À CAPSULES DOLCE GUSTO KRUPS YY1786FD,[EXPRESSO À CAPSULES DOLCE GUSTO KRUPS YY1786F...
2,ACER SWIFT SF3145139ZJ 14 CORE I36006U 2 GHZ ...,[ACER SWIFT SF3145139ZJ 14 CORE I36006U 2 GHZ ...
3,IPAD 4 128 GO WIFI NOIR,"[IPAD 4 128 GO WIFI NOIR, IPAD MINI 4 128 GO..."
4,COMPTEUR DE FANS FACEBOOK 5 CHIFFRES,"[COMPTEUR DE FANS FACEBOOK 5 CHIFFRES, HTC WIN..."
5,ASUS ZENFONE 2 SELFIE 32 GO BLEU DÉBLOQUÉ,"[ASUS ZENFONE 2 SELFIE 32 GO BLEU DÉBLOQUÉ, ..."
6,DORO 8031 8 GO NOIR DÉBLOQUÉ,"[DORO 8031 8 GO NOIR DÉBLOQUÉ, DORO 8030 8 G..."
7,VENTILATEUR DYSON COOL AM07,"[VENTILATEUR DYSON COOL AM07, VENTILATEUR DYSO..."
8,LG G6 32 GO NOIR DÉBLOQUÉ,"[LG G6 32 GO NOIR DÉBLOQUÉ, LG G2 32 GO NOI..."
9,"IPAD MINI 4 7,9'' 32 GO WIFI ARGENT","[IPAD MINI 4 7,9'' 32 GO WIFI ARGENT, IPAD ..."


-----------------------------------------------

	 Matching des produits it

-----------------------------------------------


,product,prod_correspondance
0,STAMPANTE,[STAMPANTE LASER MONOCROMATICA BROTHER HL2300D...
1,NOTE 4/5,[SAMSUNG GALAXY NOTE 4 N910F 32 GB 4G NERO S...
2,SAMSUNG NOTE 5 64GB,"[SAMSUNG GALAXY NOTE 5 64GB ORO SBLOCCATO, ..."
3,IPAD 4 64 GB WIFI + 4G,"[IPAD 4 64 GB WIFI + 4G NERO SBLOCCATO, IP..."
4,IPAD 2,"[IPAD MINI 2 16GB GRIGIO SIDERALE WIFI, IPA..."
5,MIRRORLESS SONY ALPHA 6000 BODY NERO,"[MIRRORLESS SONY ALPHA 6000 BODY NERO, MIRRO..."
6,SAMSUNG NOTE 5 64G,"[SAMSUNG GALAXY NOTE 5 64GB ORO SBLOCCATO, ..."
7,SAMSUNG,[SAMSUNG GALAXY S6 EDGE 64GB G925 4G NERO ...
8,SAMSUNG,[SAMSUNG GALAXY S6 EDGE 64GB G925 4G NERO ...
9,OBIETTIVO CANON EF 70300MM 1:4.55.6 DO IS USM,[OBIETTIVO CANON EF 70300MM 1:4.55.6 DO IS USM...
